# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from dotenv import find_dotenv
from newsapi import NewsApiClient

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/olivergeddes/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key 
api_key = "869ddd82eeae4d7ca1f649b22ca81046"

In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [6]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_everything(
    q="bitcoin OR btc",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [7]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_everything(
    q="ethereum OR eth",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [9]:
#Firstly create a function that can be applied to all currencies
def sentiment_df_creator(news):
    news_sentiment = []
    for article in news["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            # Appending the list/df
            news_sentiment.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu})
            
        except AttributeError:
            pass
  
    # Create news sentiment scores df
    news_df = pd.DataFrame(news_sentiment)

    # Reorder df
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    news_df = news_df[cols]

    return news_df

In [13]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin__sentiment_df = sentiment_df_creator(bitcoin_headlines)
bitcoin__sentiment_df.head()

,date,text,compound,positive,negative,neutral
0,2021-07-29,This story originally appeared on be[IN]crypto...,0.3400,0.079,0.000,0.921
1,2021-08-06,"August\r\n6, 2021\r\n3 min read\r\nThis articl...",-0.3400,0.000,0.066,0.934
2,2021-08-06,When my wife started a little garden in our ur...,-0.3400,0.000,0.061,0.939
3,2021-07-19,"Like Dogecoin devotees, the mayor of Reno, and...",0.6908,0.178,0.000,0.822
4,2021-08-10,Photo by Joe Raedle/Getty Images\r\n\n \n\n Tw...,0.4019,0.080,0.000,0.920


In [14]:
# Create the Ethereum sentiment scores DataFrame
Ethereum__sentiment_df = sentiment_df_creator(ethereum_headlines)
Ethereum__sentiment_df.head()

,date,text,compound,positive,negative,neutral
0,2021-07-16,There are many blockchain platforms competing ...,0.3612,0.075,0.000,0.925
1,2021-07-29,Blockchain infrastructure startups are heating...,-0.2411,0.000,0.061,0.939
2,2021-08-05,Cent was founded in 2017 as an ad-free creator...,0.6956,0.190,0.000,0.810
3,2021-08-10,"August\r\n10, 2021\r\n4 min read\r\nThis story...",0.5106,0.150,0.060,0.790
4,2021-08-10,"After the bell today, Coinbase reported anothe...",0.5719,0.139,0.000,0.861


In [16]:
# Describe the Bitcoin Sentiment
bitcoin__sentiment_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.00000,100.000000,100.000000
mean,0.073579,0.05133,0.033840,0.914850
std,0.358673,0.05974,0.051586,0.072346
min,-0.886000,0.00000,0.000000,0.724000
25%,-0.058000,0.00000,0.000000,0.867750
50%,0.000000,0.04600,0.000000,0.921000
75%,0.381800,0.08225,0.069250,1.000000
max,0.883400,0.27400,0.271000,1.000000


In [18]:
# Describe the Ethereum Sentiment
Ethereum__sentiment_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.047056,0.048170,0.032680,0.919120
std,0.364703,0.059511,0.050142,0.072508
min,-0.757900,0.000000,0.000000,0.737000
25%,-0.208300,0.000000,0.000000,0.865500
50%,0.000000,0.000000,0.000000,0.926500
75%,0.323650,0.076000,0.067500,1.000000
max,0.840200,0.237000,0.173000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: BITCOIN

Q: Which coin had the highest compound score?

A: BITCOIN

Q. Which coin had the highest positive score?

A: BITCOIN

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [33]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/olivergeddes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/olivergeddes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/olivergeddes/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [35]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_supp = {'said', 'basically', 'also', 'char', 'ha', 't'}

In [36]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
   
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    # Remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in sw.union(sw_supp)]
    
    return tokens

In [38]:
# Create a new tokens column for Bitcoin
tokens = []

for text in bitcoin__sentiment_df["text"]:
    tokens.append(tokenizer(text))

bitcoin__sentiment_df["tokens"] = tokens
bitcoin__sentiment_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-07-29,This story originally appeared on be[IN]crypto...,0.3400,0.079,0.000,0.921,"[story, originally, appeared, beincryptomaryla..."
1,2021-08-06,"August\r\n6, 2021\r\n3 min read\r\nThis articl...",-0.3400,0.000,0.066,0.934,"[august, min, readthis, article, wa, translate..."
2,2021-08-06,When my wife started a little garden in our ur...,-0.3400,0.000,0.061,0.939,"[wife, started, little, garden, urban, backyar..."
3,2021-07-19,"Like Dogecoin devotees, the mayor of Reno, and...",0.6908,0.178,0.000,0.822,"[like, dogecoin, devotee, mayor, reno, leader,..."
4,2021-08-10,Photo by Joe Raedle/Getty Images\r\n\n \n\n Tw...,0.4019,0.080,0.000,0.920,"[photo, joe, raedlegetty, images, twitter, squ..."


In [39]:
# Create a new tokens column for Ethereum
tokens = []

# Iterate through each row of tokenised text
for text in Ethereum__sentiment_df["text"]:
    tokens.append(tokenizer(text))

# Add the data to the DataFrame
Ethereum__sentiment_df["tokens"] = tokens
Ethereum__sentiment_df.tail()

,date,text,compound,positive,negative,neutral,tokens
95,2021-08-04,"A man wearing a protective face mask, followin...",0.0000,0.000,0.0,1.000,"[man, wearing, protective, face, mask, followi..."
96,2021-08-04,"NEW YORK, Aug 4 (Reuters) - The dollar recover...",0.5423,0.127,0.0,0.873,"[new, york, aug, reuters, dollar, recovered, q..."
97,2021-08-04,A U.S. dollar banknote is seen in this illustr...,0.3400,0.074,0.0,0.926,"[us, dollar, banknote, seen, illustration, tak..."
98,2021-08-05,"NEW YORK/LONDON, Aug 5 (Reuters) - The dollar ...",0.4019,0.083,0.0,0.917,"[new, yorklondon, aug, reuters, dollar, took, ..."
99,2021-08-05,U.S. One dollar banknotes are seen in front of...,0.2960,0.068,0.0,0.932,"[us, one, dollar, banknote, seen, front, displ..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [40]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [42]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [43]:
# Generate the Bitcoin word cloud
btc_wordcloud = WordCloud(colormap="RdYlBu").generate(str(btc_tokens))
plt.imshow(btc_wordcloud)

NameError: name 'btc_tokens' is not defined

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---